In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import gc
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Load Data

In [2]:
#Load Data
transactions = pd.read_csv("./transactions_train.csv") # 50 Mil # 3,5 GB # 2018-2020
articles = pd.read_csv("./articles.csv") # 
customers = pd.read_csv("./customers.csv")

# Association Rule Mining

## Preprocessing

### Sampling

In [6]:
#Reduce dataset for better perfomance
#First 3 months
# trans_3_months = transactions[transactions['t_dat'] < '2018-12-31']
# trans_3_months.sort_values(by='t_dat',ascending = False).head()
# trans_3_months.count()
# print(len(trans_3_months))

trans_3_months = transactions[transactions['t_dat'].map(lambda x: x < '2018-12-31')]
trans_3_months.sort_values(by='t_dat',ascending = False).head()
print(len(trans_3_months))

# Join Transactions and articles
join_trans = pd.merge(trans_3_months,articles,on="article_id", how="left")
# Clean table
#result = join_trans.filter(['t_dat', 'customer_id','article_id', 'product_code', 'prod_name', 'product_type_no','product_type_name', 'product_group_name','garment_group_name','section_name','index_group_name'])


4387359


### Making data ready for Mining algorithm

##### Level- Article_ID 

In [15]:
# Process for product_group_name
items_article = []
items_trans = []
items_trans.append(join_trans['article_id'][0])
for i in range(len(join_trans)-1):
    if (trans_3_months['t_dat'][i] == join_trans['t_dat'][i+1]) and (join_trans['customer_id'][i] == join_trans['customer_id'][i+1]):
        items_trans.append(join_trans['article_id'][i+1])
    else:
        items_article.append(items_trans)
        items_trans = []
        items_trans.append(join_trans['article_id'][i+1])

##### Level TYPE-  Product_type_name

In [16]:
items_type = []
items_trans = []
items_trans.append(join_trans['product_type_name'][0])
for i in range(len(join_trans)-1):
    if (trans_3_months['t_dat'][i] == join_trans['t_dat'][i+1]) and (join_trans['customer_id'][i] == join_trans['customer_id'][i+1]):
        items_trans.append(join_trans['product_type_name'][i+1])
    else:
        items_type.append(items_trans)
        items_trans = []
        items_trans.append(join_trans['product_type_name'][i+1])

##### Level Category-  index_group_name

In [9]:
items_category = []
items_trans = []
items_trans.append(join_trans['index_group_name'][0])
for i in range(len(join_trans)-1):
    if (trans_3_months['t_dat'][i] == join_trans['t_dat'][i+1]) and (join_trans['customer_id'][i] == join_trans['customer_id'][i+1]):
        items_trans.append(join_trans['index_group_name'][i+1])
    else:
        items_category.append(items_trans)
        items_trans = []
        items_trans.append(join_trans['index_group_name'][i+1])

In [17]:
print(items_article[:10])
print(items_type[:10])
print(items_category[:10])

[[663713001, 541518023], [505221004, 685687003, 685687004, 685687001, 505221001], [688873012, 501323011, 598859003, 688873020, 688873011], [531310002, 529841001], [501820043, 501820043, 674681001, 671505001, 671505001, 631848002, 631848002, 631848002, 631848002, 680187001, 676827002, 676827002, 685687002, 685687002, 680912006, 680912006, 692454002, 692454002, 640639001, 640639001, 664421002, 664421002, 680912009, 680912009, 553139001, 553139001, 377277001, 377277001, 700819006, 700819006], [397068015], [652075001, 670295001], [631744002, 562252035], [649356002, 579941002, 629760002, 625229004], [688545001]]
[['Underwear body', 'Bra'], ['Sweater', 'Sweater', 'Sweater', 'Sweater', 'Sweater'], ['Blouse', 'Trousers', 'Trousers', 'Blouse', 'Blouse'], ['Cardigan', 'Sweater'], ['Sweater', 'Sweater', 'Vest top', 'Blouse', 'Blouse', 'Skirt', 'Skirt', 'Skirt', 'Skirt', 'Vest top', 'T-shirt', 'T-shirt', 'Sweater', 'Sweater', 'T-shirt', 'T-shirt', 'Top', 'Top', 'Vest top', 'Vest top', 'Vest top', 

## Rule Mining


In [18]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=8aa5935a02f9c5b4633740bf81981ca2b5dd58a9e6d2975a34db0fa80a5bc092
  Stored in directory: c:\users\timwy\appdata\local\pip\cache\wheels\32\2a\54\10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori


## Apply Apriori

### Article Level

In [27]:
#### Level ARTICLE ID
from apyori import apriori
# items_article
## items_type
### items_category
association_rules_article = apriori(items_article, min_support=0.005, min_confidence=0.2, min_lift=1.1, min_length=2)
association_results_article = list(association_rules_article)

In [ ]:
print(len(association_results))
print(association_results[0])

In [28]:
from apyori import apriori
# items_article
## items_type
### items_category
association_rules_type = apriori(items_type, min_support=0.005, min_confidence=0.2, min_lift=1.1, min_length=2)
association_results_type = list(association_rules_type)

In [29]:
from apyori import apriori
# items_article
## items_type
### items_category
association_rules_category = apriori(items_category, min_support=0.005, min_confidence=0.2, min_lift=1.1, min_length=2)
association_results_category = list(association_rules_category)

In [25]:
print(len(association_results))
print(association_results[0])

57
RelationRecord(items=frozenset({'Swimwear bottom', 'Bikini top'}), support=0.009791988468227064, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bikini top'}), items_add=frozenset({'Swimwear bottom'}), confidence=0.7331915131836757, lift=53.276458906745056), OrderedStatistic(items_base=frozenset({'Swimwear bottom'}), items_add=frozenset({'Bikini top'}), confidence=0.7115227902428424, lift=53.27645890674506)])


##### Printing Rules

In [26]:
for rule in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = rule[0] 
    rules = [x for x in pair]
    if len(rules) > 1:
        print("Rule: " + rules[0] + " -> " + rules[1])

        #second index of the inner list
        print("Support: " + str(rule[1]))

        #third index of the list located at 0th
        #of the third index of the inner list

        print("Confidence: " + str(rule[2][0][2]))
        print("Lift: " + str(rule[2][0][3]))
        print("=====================================")

Rule: Swimwear bottom -> Bikini top
Support: 0.009791988468227064
Confidence: 0.7331915131836757
Lift: 53.276458906745056
Rule: Blazer -> Trousers
Support: 0.013672298474342853
Confidence: 0.3632409025901796
Lift: 1.333911819413938
Rule: Blouse -> Trousers
Support: 0.034603542969760495
Confidence: 0.3238770354716136
Lift: 1.1893578134281748
Rule: Underwear bottom -> Bra
Support: 0.026463863771960385
Confidence: 0.36042237315059006
Lift: 5.87714542372598
Rule: Cardigan -> Sweater
Support: 0.01901277137871377
Confidence: 0.45379110976284376
Lift: 1.3611524954693366
Rule: Cardigan -> Trousers
Support: 0.013309687616353226
Confidence: 0.3176716215492782
Lift: 1.1665699750643386
Rule: Skirt -> Dress
Support: 0.013693232709443283
Confidence: 0.2310982688134053
Lift: 1.6985049081499013
Rule: Dress -> Underwear Tights
Support: 0.0054600971049448156
Confidence: 0.20387482203176904
Lift: 1.4984205102321415
Rule: Hoodie -> Sweater
Support: 0.02575658568606722
Confidence: 0.43693322341302554
Lift: